**The Graph Structure of Public Software Development**

Antoine Pietri(1), Guillaume Rousseau(2) and Stefano Zacchiroli(3)

1. Inria, Paris, France. antoine.pietri@inria.fr
2. Université de Paris, Paris, France. guillaume.rousseau@u-paris.fr
3. LTCI, Télécom Paris, Institut Polytechnique de Paris, Paris, France. stefano.zacchiroli@telecom-paris.fr

**TODO: Add citation string**

# Replication Package : Quality and Data Integrity

In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import tabulate
from IPython.display import HTML, display
import json

import common


DATASET = Path('../experiments')

def d(p):
    x, y = common.load_text_distribution(p)
    return common.Distribution(x, y, '', '', '')

distributions = {
    'In-degrees': [
        ("Full", d(DATASET / 'inout/full_in.txt')),
        ("Filesystem", d(DATASET / 'inout/dir+cnt_in.txt')),
        ("Commit", d(DATASET / 'inout/rev_in.txt')),
        ("History", d(DATASET / 'inout/rel+rev_in.txt')),
        ("Hosting", d(DATASET / 'inout/ori+snp_in.txt')),
    ],
    'Out-degrees': [
        ("Full", d(DATASET / 'inout/full_out.txt')),
        ("Filesystem", d(DATASET / 'inout/dir+cnt_out.txt')),
        ("Commit", d(DATASET / 'inout/rev_out.txt')),
        ("History", d(DATASET / 'inout/rel+rev_out.txt')),
        ("Hosting", d(DATASET / 'inout/ori+snp_out.txt')),
    ],
    'Connected components': [
        ("Full", d(DATASET / 'connectedcomponents/full/distribution.txt')),
        ("Filesystem", d(DATASET / 'connectedcomponents/dir+cnt/distribution.txt')),
        ("Commit", d(DATASET / 'connectedcomponents/rev/distribution.txt')),
        ("History", d(DATASET / 'connectedcomponents/rel+rev/distribution.txt')),
        ("Hosting", d(DATASET / 'connectedcomponents/ori+snp/distribution.txt')),
    ],
    'Clustering coefficient': [
        ("Full", d(DATASET / 'clusteringcoeff/distribution-full.txt')),
        ("Filesystem", d(DATASET / 'clusteringcoeff/distribution-dircnt.txt')),
        ("Commit", d(DATASET / 'clusteringcoeff/distribution-rev.txt')),
        ("History", d(DATASET / 'clusteringcoeff/distribution-relrev.txt')),
        # ("Hosting", d(DATASET / 'clusteringcoeff/distribution-orisnp.txt')),
    ],
    'Shortest path': [
        ("Filesystem", d(DATASET / 'shortestpath/dir+cnt/distribution.txt')),
        ("Commit", d(DATASET / 'shortestpath/rev/distribution.txt')),
    ]
}

## Graph statistics (node and edge types)

In [2]:
node_stats = {k: int(v) for k, v in [l.split() for l in (DATASET / 'stats/graph.nodes.stats.txt').read_text().strip().split('\n')]}
edge_stats = {k: int(v) for k, v in [l.split() for l in (DATASET / 'stats/graph.edges.stats.txt').read_text().strip().split('\n')]}
node_count = sum(node_stats.values())
edge_count = sum(edge_stats.values())

display(HTML(tabulate.tabulate(
    [[common.types_verbose[k], v, v / node_count * 100] for k, v in node_stats.items()],
    headers=["Node type", "Nodes", "%"],
    tablefmt='html'
)))

display(HTML(tabulate.tabulate(
    [["{} → {}".format(common.types_verbose[k.split(':')[0]], common.types_verbose[k.split(':')[1]]), v, v / edge_count * 100] for k, v in edge_stats.items()],
    headers=["Edge type", "Edges", "%"],
    tablefmt='html'
)))

Node type,Nodes,%
contents,9152847293,47.3487
directories,7897590134,40.8551
origins,147453557,0.762793
releases,16539537,0.0855608
revisions,1976476233,10.2245
snapshots,139832772,0.72337


Edge type,Edges,%
directories → contents,149267317723,67.3929
directories → directories,64584351336,29.1593
directories → revisions,792196260,0.35767
origins → snapshots,776112709,0.350408
releases → contents,1066,4.8129e-07
releases → directories,1678,7.57603e-07
releases → releases,38219,1.72556e-05
releases → revisions,16492908,0.00744641
revisions → directories,1971187167,0.889974
revisions → revisions,2021009703,0.912469


## Per-layer distribution statistics

Statistics of the graph layers and their associated distributions, as reported in the article.

In [3]:
# it can take few minutes to process
headers = ["Algorithm", "Layer", "Number of objects", "Scaling parameter", "X decades", "Y decades"]
table = []
for algo_name, algo_distributions in distributions.items():
    for name, distribution in algo_distributions:
        row = [
            algo_name,
            name,
            f'{int(np.sum(distribution.y)):,}',
            distribution.fitted_power(),
            np.log10(np.max(distribution.x)),
            np.log10(np.max(distribution.y)),
        ]
        table.append(row)

display(HTML(tabulate.tabulate(table, headers=headers, tablefmt='html')))

Algorithm,Layer,Number of objects,Scaling parameter,X decades,Y decades
In-degrees,Full,"19,330,739,526",1.86533,8.47619,10.1453
In-degrees,Filesystem,"17,050,437,427",1.86295,8.47619,10.0273
In-degrees,Commit,"1,976,476,233",2.20457,5.84003,9.23299
In-degrees,History,"1,993,015,770",2.14762,5.84003,9.23155
In-degrees,Hosting,"287,286,329",2.76256,7.03349,8.16881
Out-degrees,Full,"19,330,739,526",1.94752,6.01419,9.96291
Out-degrees,Filesystem,"17,050,437,427",1.94683,6.01419,9.96169
Out-degrees,Commit,"1,976,476,233",5.80822,5,9.24394
Out-degrees,History,"1,993,015,770",5.80822,5,9.24802
Out-degrees,Hosting,"287,286,329",2.20614,4.98671,8.22387


## Data integrity: in and out degrees

This data helps getting an overview of the graph properties and check whether it is consistent to our expectations as a way to perform data integrity checks.

In [4]:
inout_per_type = [
    'cnt_in_dir',
    'dir_in_all',
    'dir_in_dir',
    'dir_in_rev',
    'dir_out_all',
    'dir_out_cnt',
    'dir_out_dir',
    'dir_out_rev',
    'ori_out_snp',
    'rel_in_snp',
    'rev_in_all',
    'rev_in_dir',
    'rev_in_rel',
    'rev_in_rev',
    'rev_in_snp',
    'rev_out_rev',
    'snp_in_ori',
    'snp_out_all',
    'snp_out_rel',
    'snp_out_rev',
]

headers = ["Node type", "Direction", "Neighbor type", "# Nodes", "# Edges", "Avg degree", "# (Lowest degree)", "# (Second-lowest)"]
table2 = []
for name in inout_per_type:
    dist = d(DATASET / f'inout/per_type/{name}.txt')
    src, direction, dst = name.split('_')
    row = [
        common.types_verbose[src],
        ("← in " if direction == 'in' else "→ out "),
        common.types_verbose[dst],
        f'{int(np.sum(dist.y)):,}',
        f'{int(np.sum(dist.x * dist.y)):,}',
        np.sum(dist.x * dist.y) / np.sum(dist.y),
        f'{int(dist.y[0]):,} ({int(dist.x[0]):,})',
        f'{int(dist.y[1]):,} ({int(dist.x[1]):,})',
    ]
    table2.append(row)

display(HTML(tabulate.tabulate(table2, headers=headers, tablefmt='html')))

Node type,Direction,Neighbor type,# Nodes,# Edges,Avg degree,# (Lowest degree),# (Second-lowest)
contents,← in,directories,"9,152,847,293","143,786,784,566",15.7095,"5,978,249,005 (1)","1,098,223,970 (2)"
directories,← in,everything,"7,897,590,134","65,200,402,547",8.25573,"1,343,830 (0)","6,134,767,929 (1)"
directories,← in,directories,"7,897,590,134","63,229,213,027",8.00614,"1,607,262,793 (0)","4,669,554,466 (1)"
directories,← in,revisions,"7,897,590,134","1,971,187,167",0.249594,"6,261,880,169 (0)","1,504,272,429 (1)"
directories,→ out,everything,"7,897,590,134","207,805,470,722",26.3125,"557,087 (0)","1,713,055,834 (1)"
directories,→ out,contents,"7,897,590,134","143,786,781,408",18.2064,"1,787,869,540 (0)","1,421,143,792 (1)"
directories,→ out,directories,"7,897,590,134","63,229,213,027",8.00614,"2,753,589,255 (0)","1,734,567,306 (1)"
directories,→ out,revisions,"7,897,590,134","789,473,873",0.0999639,"7,860,017,187 (0)","23,267,141 (1)"
origins,→ out,snapshots,"147,453,557","189,314,705",1.28389,"22,710,546 (0)","77,244,971 (1)"
releases,← in,snapshots,"16,539,537","700,135,072",42.331,"427,531 (0)","4,408,973 (1)"


### Criteria list
Here are a few examples of criteria that can be checked on the table:

1. The number of nodes computed from the distributions (= the sum of the second column) is always the same in all distributions starting from the same node type. For instance, `dir_in_*` and `dir_out_*` all have the same number of directory nodes which have to be equals to the number of directory nodes in the raw swh dataset (namely 7 897 590 134).
1. The number of edges computed from a source type to a destination type (src_out_dest) equals the number of edges from the raw dataset.
1. The total (or average) in/outdegree of a given object type is equal when each neighbor type is looked independently and when they are all aggregated together (e.g. the average degree of `dir_out_all` is a weighted average of the average degrees of the `dir_out_{cnt,dir,rev}` distributions).
1. The number of objects with a total indegree of 0 should be small in all types of objects that are supposed to be reachable from the upper layers of the graph.
1. The number of objects with a total outdegree of 0 should be small in specific types of objects that are supposed to reach the lower layers of the graph.
1. Some specific per-layer indegrees are expected to be relatively small compared to the total number of objects (e.g. most revisions do not have an associated release)

We check some of these criteria programatically below.

In [5]:
def checkmark(condition):
    if condition:
        return '<span style="color: green">✔</span>'
    else:
        return '<span style="color: red">✗</span>'

#### Criterion 1: y values sums up to the number of nodes in the layer

In [22]:
table = []
for name in inout_per_type:
    dist = d(DATASET / f'inout/per_type/{name}.txt')
    src, direction, dst = name.split('_')
    sum_y = int(np.sum(dist.y))
    table.append([
        name, sum_y, node_stats[src], 
        checkmark(sum_y == node_stats[src]),
        100*(sum_y-node_stats[src])/node_stats[src]
    ])
    
display(HTML(tabulate.tabulate(
    table,
    headers=["Distribution", "Nodes from distribution", "Nodes from dataset", "Result", "Difference %"],
    tablefmt='unsafehtml'
)))

Distribution,Nodes from distribution,Nodes from dataset,Result,Difference %
cnt_in_dir,9152847293,9152847293,✔,0
dir_in_all,7897590134,7897590134,✔,0
dir_in_dir,7897590134,7897590134,✔,0
dir_in_rev,7897590134,7897590134,✔,0
dir_out_all,7897590134,7897590134,✔,0
dir_out_cnt,7897590134,7897590134,✔,0
dir_out_dir,7897590134,7897590134,✔,0
dir_out_rev,7897590134,7897590134,✔,0
ori_out_snp,147453557,147453557,✔,0
rel_in_snp,16539537,16539537,✔,0


#### Criterion 2: weighted sum of values sums up to the number of edges in the layer

In [7]:
table = []
for name in inout_per_type:
    dist = d(DATASET / f'inout/per_type/{name}.txt')
    src, direction, dst = name.split('_')
    computed_edges = int(np.sum(dist.x * dist.y))
    expected_edges = sum(
        v
        for k, v in edge_stats.items()
        if k.split(':')[direction == 'in'] == src
        and (dst == 'all' or k.split(':')[direction != 'in'] == dst)
    )
    table.append([
        name,
        computed_edges,
        expected_edges,
        checkmark(computed_edges == expected_edges),
        100*abs(computed_edges - expected_edges) / expected_edges
    ])
    
display(HTML(tabulate.tabulate(
    table,
    headers=["Distribution", "Edges from distribution", "Edges from dataset", "Result", "Difference %"
            ],
    tablefmt='unsafehtml'
)))

Distribution,Edges from distribution,Edges from dataset,Result,Difference %
cnt_in_dir,143786784566,149267317723,✗,3.67162
dir_in_all,65200402547,66555540858,✗,2.0361
dir_in_dir,63229213027,64584351336,✗,2.09825
dir_in_rev,1971187167,1971187167,✔,0
dir_out_all,207805470722,214643865319,✗,3.18593
dir_out_cnt,143786781408,149267317723,✗,3.67163
dir_out_dir,63229213027,64584351336,✗,2.09825
dir_out_rev,789473873,792196260,✗,0.343651
ori_out_snp,189314705,776112709,✗,75.6073
rel_in_snp,700135072,700823546,✗,0.0982379


Here, we observe some discrepancy between the expected number of edges from the dataset and the ones we find in the distributions. As explained in the "Threats to validity/Multiple edges" section of the paper, this is due to technical limitations in the compression pipeline we have used to load the global VCS graph in-memory at the time the experiments were conducted.

The 75% and 15% decreases in edges whose source or target is a snapshot node are not the most impactful regarding the graph structure of public software development because these duplications are the result of crawling processes (characterizing SoftwareHeritage crawlers visiting these origins several times and having ”seen” these snapshots more than once).

It is potentially more so for the filsystem layer. The difference of a few percent is of the same order as some of the distribution tails we have highlighted and discussed. We have not attempted to measure the impact on the studied distributions, but this is likely to have a quantitative impact on some of the results presented in this study (such as the estimates of exponents, ranges and means, or properties of the shortest paths for instance), without necessarily calling them into question qualitatively.


#### Criterion 3.1: per-type average degrees sum up to layer average degree

In [23]:
table = []
for name in inout_per_type:
    dist = d(DATASET / f'inout/per_type/{name}.txt')
    src, direction, dst = name.split('_')
    if dst == 'all':
        reported_avg_degree = np.sum(dist.x * dist.y) / np.sum(dist.y)
        subcomponent_distributions = [
            d(DATASET / f'inout/per_type/{name}.txt')
            for name in inout_per_type
            if name.split('_')[0] == src and name.split('_')[1] == direction and name.split('_')[2] != 'all'
        ]
        computed_weighted_avg_degree = sum(
            np.sum(dist.x * dist.y) / np.sum(dist.y)
            for dist in subcomponent_distributions
        )        
        table.append([
            name,
            reported_avg_degree,
            computed_weighted_avg_degree,
            checkmark(reported_avg_degree == computed_weighted_avg_degree),
            100*abs(reported_avg_degree - computed_weighted_avg_degree) / reported_avg_degree
        ])
    
display(HTML(tabulate.tabulate(
    table,
    headers=["Distribution", "Reported average degree", "Computed average", "Result","Difference %",""],
    tablefmt='unsafehtml'
)))

Distribution,Reported average degree,Computed average,Result,Difference %
dir_in_all,8.25573,8.25573,✗,3.60887e-06
dir_out_all,26.3125,26.3125,✗,1.16166e-06
rev_in_all,2.00969,2.00969,✔,0
snp_out_all,13.2035,13.2034,✗,0.00015274


#### Criterion 3.2: per-type total degrees sum up to layer average degree

In [24]:
table = []
for name in inout_per_type:
    dist = d(DATASET / f'inout/per_type/{name}.txt')
    src, direction, dst = name.split('_')
    if dst == 'all':
        reported_total_degree = np.sum(dist.x * dist.y)
        subcomponent_distributions = [
            d(DATASET / f'inout/per_type/{name}.txt')
            for name in inout_per_type
            if name.split('_')[0] == src and name.split('_')[1] == direction and name.split('_')[2] != 'all'
        ]
        computed_weighted_total_degree = sum(
            np.sum(dist.x * dist.y)
            for dist in subcomponent_distributions
        )
        table.append([
            name,
            int(reported_total_degree),
            int(computed_weighted_total_degree),
            checkmark(reported_total_degree == computed_weighted_total_degree),
            100*abs(reported_total_degree - computed_weighted_total_degree) / reported_total_degree
        ])
    
display(HTML(tabulate.tabulate(
    table,
    headers=["Distribution", "Reported total degree", "Computed total", "Result","Difference %"],
    tablefmt='unsafehtml'
)))

Distribution,Reported total degree,Computed total,Result,Difference %
dir_in_all,65200402547,65200400194,✗,3.60887e-06
dir_out_all,207805470722,207805468308,✗,1.16166e-06
rev_in_all,3972106851,3972106851,✔,0
snp_out_all,1846275796,1846272976,✗,0.00015274


#### Criterion 4: low number of non-origin nodes with no parents

In [10]:
table = []
for name in ["cnt_in_dir", "dir_in_all", "rel_in_snp", "rev_in_all", "snp_in_ori"]:
    dist = d(DATASET / f'inout/per_type/{name}.txt')
    src, direction, dst = name.split('_')
    indegree_zero = int(dist.y[0]) if dist.x[0] == 0 else 0
    table.append([
        name,
        indegree_zero,
        node_stats[src],
        checkmark(indegree_zero==0),
        100*indegree_zero / node_stats[src],
    ])
    
display(HTML(tabulate.tabulate(
    table,
    headers=["Distribution", "Nodes with indegree = 0", "Total nodes", "Result","Percentage %"],
    tablefmt='unsafehtml'
)))

Distribution,Nodes with indegree = 0,Total nodes,Result,Percentage %
cnt_in_dir,0,9152847293,✔,0
dir_in_all,1343830,7897590134,✗,0.0170157
rel_in_snp,427531,16539537,✗,2.5849
rev_in_all,21591750,1976476233,✗,1.09244
snp_in_ori,53736,139832772,✗,0.0384288


The list of nodes without ancestors in this study will be made available later in a deposit on zenodo (**TODO: add link**) 

#### Criterion 5: low number of intermediate objects with no children

In [11]:
table = []
for name in ["dir_out_all", "snp_out_all", "ori_out_snp"]:
    dist = d(DATASET / f'inout/per_type/{name}.txt')
    src, direction, dst = name.split('_')
    if direction != 'out':
        continue
    indegree_zero = int(dist.y[0]) if dist.x[0] == 0 else 0
    table.append([
        name,
        indegree_zero,
        node_stats[src],
        checkmark(indegree_zero==0),
        100*indegree_zero / node_stats[src]
    ])
    
display(HTML(tabulate.tabulate(
    table,
    headers=["Distribution", "Nodes with indegree = 0", "Total nodes", "Result", "Percentage %"],
    tablefmt='unsafehtml'
)))

Distribution,Nodes with indegree = 0,Total nodes,Result,Percentage %
dir_out_all,557087,7897590134,✗,0.00705389
snp_out_all,43567,139832772,✗,0.0311565
ori_out_snp,22710546,147453557,✗,15.4018


We see here that a lot of origins exist but do not have associated snapshots. This may be due to the crawling process if for instance 
- they have been listed in the dataset but not been visited yet by the crawlers, 
- if the first crawling failed before completing, or
- if they have been removed before the crawlers visited them.

Some of the strategies described in the next section for nodes without ancestors may highlight one or more of these possible explanations.

#### Criterion 6: low number of revision nodes with ancestor release

In [16]:
DATASET = Path('../experiments')
table = []

threshold=5 # %
# Threshold value meaning "small compared to the number of nodes" 
# based on an a priori knowledge of the object of study (here the VCS history graph) 
# independently of the dataset specifically taken for the study (here the one made 
# available by the software heritage project)

for name in ['rev_in_rel']:
    dist = d(DATASET / f'inout/per_type/{name}.txt')
    src, direction, dst = name.split('_')
    sy=np.sum(dist.y)
    if dist.x[0]==0:
        s0=dist.y[0]
    else:
        s0=0
    ds=(sy-s0)/sy*100
    table.append([
        name,
        int(sy-s0),
        int(sy),
        checkmark(ds<threshold),
        ds
    ])
    
display(HTML(tabulate.tabulate(
    table,
    headers=["Distribution", "Rev. with ancestor rel.", "Total nodes", "Result (<"+str(threshold)+"%)", "Percentage %"],
    tablefmt='unsafehtml'
)))

Distribution,Rev. with ancestor rel.,Total nodes,Result (<5%),Percentage %
rev_in_rel,11722969,1976476233,✔,0.593125


## Example of data integrity failures encountered during this study


### Data integrity: nodes without ancestors / Compression Pipline

The control script for criteria 5 applied on the dataset 2020-05-20 
(https://annex.softwareheritage.org/public/dataset/graph/2020-05-20/compressed/) 
lead to the following result:

In [17]:
DATASET_20200520 = Path('../experiments/deprecated/20210403/') # after bugfix

table = []
for name in ["snp_in_ori","cnt_in_dir", "dir_in_all", "rel_in_snp", "rev_in_all"]:
    dist = d(DATASET_20200520 / f'inout/per_type/{name}.txt')
    src, direction, dst = name.split('_')
    indegree_zero = int(dist.y[0]) if dist.x[0] == 0 else 0
    table.append([
        name,
        indegree_zero,
        node_stats[src],
        checkmark(indegree_zero == node_stats[src]),
        100*indegree_zero / node_stats[src],
    ])
    
display(HTML(tabulate.tabulate(
    table,
    headers=["Distribution", "Nodes with indegree = 0", "Total nodes", "Result","Percentage %" ],
    tablefmt='unsafehtml'
)))

Distribution,Nodes with indegree = 0,Total nodes,Result,Percentage %
snp_in_ori,51899904,139832772,✗,37.1157
cnt_in_dir,302918865,9152847293,✗,3.30956
dir_in_all,1968810,7897590134,✗,0.0249293
rel_in_snp,428698,16539537,✗,2.59196
rev_in_all,17852476,1976476233,✗,0.903248


**In this older export, 37% of the snaphost nodes were not connected to an upstream origins.** This control script allowed us to spot the issue, and shift to a newer version of the dataset (2020-12-15) in which (see criteria 4 script) we now have less thant 0.04% of snapshot nodes without ancestor origins. Directory nodes without ancestors decrease from 3.3% to less than 0.02%.

### Data integrity: nodes without ancestors / Raw Dataset

Due to the update mechanisms of the software heritage project base, having parents without ancestors can have several origins. One of them is the atomicity that is not guaranteed during an update in the bottom-up direction. That is, if there is a problem in the indexing process of new software artifacts, the nodes of the filesystem layer, for example, may have been injected without the nodes of the history or hosting layers having been injected.
In most cases, when crawlers return to an origin whose last visit was an error, the injection process based on intrinsic identifiers corrects the problem.

Similarly, the process, used up to now, to export the graph and build a compressed version is not atomic. So there may be a time shift of the same type with some objects missing at the frontier of the current injection processes.

In both cases, the problems should be only temporary and it is possible to check whether this explains all or some of the nodes without ancestors we have found, by 
- comparing two exports separated by a time guaranteeing that the crawlers have returned to the failed visits, and checking that most nodes missing an ancestor in the first export, have one in the second export.
- comparing the 2020-12-15 export with the information contained in the Software Heritage project database. In the case of revisions not linked to an ancestor, it is sufficient to check whether these revisions were seen in visits much older than the export date, or on the contrary close to the limit represented by the export date.

We did this on a sample of 1000 revisions without ancestors, identifying for 98.5% of them (see file *rev1000.txt*) the oldest visit in which it was seen (without having to go back in the chain of revisions).

A small proportion of these revisions have been seen recently. This invalidates the hypothesis according to which nodes without ancestors are primarily caused by the non-atomicity of the crawling process and the export process.

Further investigation is needed. An anomaly report has been filed https://forge.softwareheritage.org/T3660.

At this point, we have no evidence that these anomalies have a significant impact on the results presented in this study. Nevertheless, this is one of the limitations of this study, and will need further investigation.




## Data Availability Statements

According to [Data Policy and Data Availability Statements ](https://www.springer.com/journal/10664/submission-guidelines#Instruction%20for%20Authors_Research%20Data%20Policy%20and%20Data%20Availability%20Statements), 
- the main datasets generated during and/or analysed during the current study are available in the https://github.com/seirl/swh-graph-structure repository,
- intermediate datasets obtained in the normal course of development of the analysis tools are not available.






